### Example-1 
### Saving variables only. Save weights and bias Tensors.

In [1]:
import tensorflow as tf

# The file path to save the data
save_file = 'model.ckpt'

# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

with tf.Session() as sess:
    # Initialize all the Variables
    sess.run(tf.initialize_all_variables())

    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

    # Save the model
    saver.save(sess, save_file)

Weights:
[[ 0.34122056  0.4240528  -0.5166108 ]
 [-0.47159606  1.05553317 -0.21383822]]
Bias:
[-1.66044831  0.30295652  0.30859965]


### Loading variables.
You still need to create the weights and bias Tensors in Python.

In [2]:
# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

with tf.Session() as sess:
    # Load the weights and bias
    saver.restore(sess, save_file)

    # Show the values of weights and bias
    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

Weight:
[[ 0.34122056  0.4240528  -0.5166108 ]
 [-0.47159606  1.05553317 -0.21383822]]
Bias:
[-1.66044831  0.30295652  0.30859965]


### Example-2
### Save an entire trained model. 

### Import mnist data. 

In [3]:
# Remove previous Tensors and Operations
tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

# Import MNIST data
mnist = input_data.read_data_sets('.', one_hot=True)

print("Import data done.")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz
Import data done.


### Create variables and the computation graph.

In [4]:
learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]), name='weights_0')
bias = tf.Variable(tf.random_normal([n_classes]), name='bias_0')

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


### Train the model and save the weights.

In [5]:
import math

save_file = 'train_model.ckpt'
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)

        # Loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict={features: batch_features, labels: batch_labels})

        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))

    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.0982000008225441
Epoch 10  - Validation Accuracy: 0.26339998841285706
Epoch 20  - Validation Accuracy: 0.4090000092983246
Epoch 30  - Validation Accuracy: 0.4991999864578247
Epoch 40  - Validation Accuracy: 0.5623999834060669
Epoch 50  - Validation Accuracy: 0.605400025844574
Epoch 60  - Validation Accuracy: 0.6367999911308289
Epoch 70  - Validation Accuracy: 0.6588000059127808
Epoch 80  - Validation Accuracy: 0.6805999875068665
Epoch 90  - Validation Accuracy: 0.6976000070571899
Trained Model Saved.


### Load a Trained Model
Let's load the weights and bias from memory, then check the test accuracy. We can change save_file to restore another saved model. For example, now save_file is train_model.ckpt. We can let save_file = 'newname_model.ckpt'. 

In [6]:
save_file

'train_model.ckpt'

In [7]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: mnist.test.images, labels: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

Test Accuracy: 0.7235999703407288


### Load weights and bias from the saved model. 

In [8]:
save_file

'train_model.ckpt'

### In the train_model.ckpt, 
we name weights and bias as 'weights_0' and 'bias_0'. Hence to load the same variables, we have to assign the name to weights and bias Tensors, otherwise there is name error 'name not found'. 

In [9]:
import tensorflow as tf
# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([784, 10]), name='weights_0')
bias = tf.Variable(tf.truncated_normal([10]), name='bias_0')

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Load Weights: {}'.format(weights.name))
print('Load Bias: {}'.format(bias.name))

with tf.Session() as sess:
    # Load the weights and bias
    saver.restore(sess, save_file)
    print('Load Weights: {}'.format(weights.name))
    print('Load Bias: {}'.format(bias.name))

print("Load weights and bias is done.")   

Load Weights: weights_0:0
Load Bias: bias_0:0
Load Weights: weights_0:0
Load Bias: bias_0:0
Load weights and bias is done.
